In [15]:
# Importamos las bibliotecas necesarias
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import gausspy.gp as gp
from scipy.optimize import curve_fit

In [16]:
# Cargamos datos desde archivos CSV para observaciones 'hot' y 'cold'
hot_data = pd.read_csv('hot_data.csv').values
cold_data = pd.read_csv('cold_data.csv').values

# Realizamos un análisis y una visualización básica de los datos
plt.plot(hot_data[:, 0], hot_data[:, 1], label='Hot Data')
plt.plot(cold_data[:, 0], cold_data[:, 1], label='Cold Data')
plt.legend()
plt.show()

# Ejemplo de calibración utilizando los datos 
calibrated_data = (hot_data[:, 1] - cold_data[:, 1]) / cold_data[:, 1]

# Guardamos los datos en un archivo CSV
pd.DataFrame({'Frequency': hot_data[:, 0], 'CalibratedIntensity': calibrated_data}).to_csv('calibrated_data.csv', index=False)

In [17]:
def load_and_plot_spectrum(file_path):
    """
    Carga y grafica un espectro desde un archivo CSV.

    Parámetros:
    file_path (str): Ruta al archivo CSV que contiene los datos del espectro.
    """
    spectrum = pd.read_csv(file_path).values
    plt.plot(spectrum[:, 0], spectrum[:, 1])
    plt.title(f'Spectrum: {file_path}')
    plt.show()

# Directorio con los archivos de espectro
directory = 'data_directory'
# Iteramos a través de los archivos en el directorio y graficar cada espectro
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        load_and_plot_spectrum(os.path.join(directory, filename))


In [18]:
#Calibración simple
def calibrate_spectrum(spectrum):
    """
    Calibra un espectro dividiendo cada punto por el máximo valor de intensidad.

    Parámetros:
    spectrum (array): Espectro a calibrar.

    Retorna:
    array: Espectro calibrado.
    """
    # Ejemplo de calibración
    return spectrum[:, 1] / np.max(spectrum[:, 1])

# Cargamos un espectro desde un archivo CSV
spectrum = pd.read_csv('spectrum.csv').values
# Calibramos el espectro
calibrated_spectrum = calibrate_spectrum(spectrum)

# Guardamos el espectro calibrado
pd.DataFrame({'Frequency': spectrum[:, 0], 'CalibratedIntensity': calibrated_spectrum}).to_csv('calibrated_spectrum.csv', index=False)


In [19]:
#Ejemplo de un ajuste lineal para la linea de base
def fit_baseline(spectrum):
    """
    Ajusta una línea de base lineal a un espectro y la resta del mismo.

    Parámetros:
    spectrum (array): Espectro del cual se ajustará y restará la línea de base.

    Retorna:
    array: Espectro con la línea de base restada.
    """
    x = spectrum[:, 0]
    y = spectrum[:, 1]
    fit = np.polyfit(x, y, 1)
    return np.polyval(fit, x)

# Cargamos el espectro calibrado desde un archivo CSV
spectrum = pd.read_csv('calibrated_spectrum.csv').values
# Ajustamos y restamos la línea de base
baseline = fit_baseline(spectrum)
corrected_spectrum = spectrum[:, 1] - baseline

# Guardamos el espectro corregido
pd.DataFrame({'Frequency': spectrum[:, 0], 'CorrectedIntensity': corrected_spectrum}).to_csv('corrected_spectrum.csv', index=False)



In [20]:
#Ejemplo básico para poder acercarnos al ajuste de la linea de 21 cm
# Definimos una función gaussiana para el ajuste de los espectros
def gaussian(x, amp, cen, wid):
     """
    Función gaussiana.

    Parámetros:
    x (array): Valores independientes.
    amp (float): Amplitud de la gaussiana.
    cen (float): Centro de la gaussiana.
    wid (float): Ancho de la gaussiana.

    Retorna:
    array: Valores de la función gaussiana aplicada a x.
    """
    return amp * np.exp(-(x-cen)**2 / (2*wid**2))

def analyze_spectrum(file_path):
    """
    Analiza un espectro cargado desde un archivo CSV y realiza un ajuste gaussiano.

    Parámetros:
    file_path (str): Ruta al archivo que contiene los datos del espectro.

    Retorna:
    tuple: Parámetros del ajuste gaussiano.
    """
    spectrum = pd.read_csv(file_path).values
    x = spectrum[:, 0]
    y = spectrum[:, 1]
    
     # Realizamos un ajuste gaussiano
    popt, _ = curve_fit(gaussian, x, y, p0=[1, 0, 2])

    return popt

# Cargamos espectros y realizamos análisis para obtener velocidades
spectral_velocities = []
directory = 'data_directory'
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        popt = analyze_spectrum(os.path.join(directory, filename))
        spectral_velocities.append(popt[1])  # Asumimos que el centro de la gaussiana es la velocidad

# Cargamos datos de distancia y velocidad desde un archivo CSV
data = pd.read_csv('rotational_data.csv')

# Separamos las distancias y velocidades del archivo CSV
distances = data['Distance']
velocities = data['Velocity']

# Graficamos la curva de rotación
plt.figure(figsize=(10, 6))
plt.plot(distances, velocities, 'o-', label='Curva de Rotación desde CSV')
plt.plot(distances, spectral_velocities, 'x-', label='Curva de Rotación desde Análisis Espectral')
plt.xlabel('Distancia al Centro Galáctico (kpc)')
plt.ylabel('Velocidad (km/s)')
plt.title('Comparación de Curvas de Rotación Galáctica')
plt.legend()
plt.grid(True)
plt.show()